# Benchmarks

When submitting a benchmark, you must submit a lambda function that expects exactly one input. This input is a list: the first element is a `ProteinConformer` and the second 
a `Smol`

In [2]:
# |hide
from rush import build_blocking_provider

In [3]:
# |hide
# hidden setup for the notebook
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/benchmark_notebook").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR, exist_ok=True)
# swap into clean workdir so that our tests are deterministic
os.chdir(WORK_DIR)
PUT_YOUR_TOKEN_HERE = os.environ["RUSH_TOKEN"]
PUT_YOUR_PREFERRED_WORKING_DIRECTORY_HERE = WORK_DIR
os.environ["RUSH_RESTORE_BY_DEFAULT"] = "False"

In [4]:
# |hide
client = build_blocking_provider(
    access_token=PUT_YOUR_TOKEN_HERE
    # for example, if your token is 00000000-dddd-cccc-0000-11111111,
    # then you should put access_token="00000000-dddd-cccc-0000-11111111"
    # (including the double quotes)
)

2025-02-07 20:36:08,501 - rush - INFO - Not restoring by default via env


In [5]:
# |hide
benchmark = client.benchmark(name="OpenFF Protein-Ligand Binding Benchmark")

In [6]:
# |hide
from IPython.display import Markdown as md
rex_code_above = """
(λ input → 
    let        
        {- protein conformers are a single physical shape of a protein -}
        protein_conformer = load (id (get 0 input)) "ProteinConformer",

        {- protein is a description of the protein e.g. its amino acid sequence -}
        protein = load (protein_id protein_conformer) "Protein",

        {- smol is a description of a small molecule e.g. its SMILES string -}
        smol = load (id (get 1 input)) "Smol",

        {- TODO(you): this is where you need to write code that predicts the binding affinity -}
        affinity = 0.5

    in
        {- the output must be a list with exactly one element: our binding affinity prediction for this input -}
        [BenchmarkArg {
            entity = "BindingAffinity", 
            id = save (BindingAffinity {
                affinity = affinity,
                affinity_metric = "kcal/mol",
                protein_id = id protein,
                smol_id = id smol,
                metadata = Metadata {
                    name = "binding affinity for smol and protein",
                    description = none,
                    tags = []
                }
            })
        }]
)
"""

In [7]:
#| echo:false
md(f"```haskell{rex_code_above}```")

```haskell
(λ input → 
    let        
        {- protein conformers are a single physical shape of a protein -}
        protein_conformer = load (id (get 0 input)) "ProteinConformer",

        {- protein is a description of the protein e.g. its amino acid sequence -}
        protein = load (protein_id protein_conformer) "Protein",

        {- smol is a description of a small molecule e.g. its SMILES string -}
        smol = load (id (get 1 input)) "Smol",

        {- TODO(you): this is where you need to write code that predicts the binding affinity -}
        affinity = 0.5

    in
        {- the output must be a list with exactly one element: our binding affinity prediction for this input -}
        [BenchmarkArg {
            entity = "BindingAffinity", 
            id = save (BindingAffinity {
                affinity = affinity,
                affinity_metric = "kcal/mol",
                protein_id = id protein,
                smol_id = id smol,
                metadata = Metadata {
                    name = "binding affinity for smol and protein",
                    description = none,
                    tags = []
                }
            })
        }]
)
```

In [8]:
#| hide
import re

def remove_comments(text):
    # Pattern explanation:
    # \{-    : matches the opening '{-'
    # .*?    : matches any characters (non-greedy)
    # -\}    : matches the closing '-}'
    pattern = r'\{-.*?-\}'
    
    # Use re.sub to replace all matches with empty string
    # re.DOTALL flag allows . to match newlines
    return re.sub(pattern, '', text, flags=re.DOTALL)

# Example usage
text = '''let a = 1, {- some comment -}
{-another comment -}
b = 2,
in a'''

cleaned_rex = remove_comments(rex_code_above)

In [9]:
#|hide
submission = client.run_benchmark(
    benchmark.id, 
    cleaned_rex, 
    "hardcoded submission", 
    sample=0.2
)

View your submission at https://rush.cloud/project/f18faf61-5556-4a78-b983-e85a3c975fa6/runs?selectedRunId=9e99fda9-6cd9-466f-8e55-40e5c8241572
